# FXa: Model Selection

In [1]:
import pandas as pd
import numpy as np
import glob, sys, os
sys.path.append('..')

In [2]:
from modules.plotting_metrics import PlotMetric
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', context='talk', font_scale=0.8)

In [3]:
file_name = './df_DkSc_results_COCRYS_DEKOIS_DUD.pkl'
X_merged_dksc = pd.read_pickle(file_name)
# Extract activity column
y_true_merged = X_merged_dksc['activity']
# Drop column from merged_dkksc
X_merged_dksc = X_merged_dksc.drop('activity', axis=1)
X_merged_dksc.shape
y_true_merged.loc['DEKOIS'].sum()

40

### Timeout Decorator

In [4]:
import signal
from functools import wraps

def timeout(n_seconds=300):
    '''Stops a function execution after n seconds'''
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Set alarm for n seconds
            signal.alarm(n_seconds)
            try:
                # Call decorated func
                return func(*args, **kwargs)
#             except TimeoutError as e:
                print(f'Execution finished after {n_seconds}:', e)
            finally:
                # Cancel Alarm
                signal.alarm(0)
        return wrapper
    return decorator

## Scaffold Splitting

In [5]:
#*************************************************
# Functions to compute stratify scaffold splitting
#*************************************************
from scaffold_splitter import train_test_scaffold_split

In [6]:
# Compute or load the dataframe containing the Generic Murcko Scaffolds
file = './df_COCRYS_DUD_DEKOIS_Murcko_Scaffolds_SMILES.obj'

df_scff_murcko = pd.read_pickle(file)

## Train/test on the same dataset 

### Learning Curves

In [7]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve

In [8]:
def plot_learning_curves(estimator, X, y, title, ylim=[0.5,1], axes=None,
                         cv=3, train_sizes=np.linspace(0.1, 1.0, 10), 
                         scoring='roc_auc', n_jobs=4):
    '''
    Plot estimator performance on the training and validation
    sets as a function of the training set size.

    Parameters
    ----------
    estimator: sklearn estimator object type
       Object type that implements the "fit" and "predict method"

    X: array-like, shape (m_samples, n_features)
        Training array with m_samples and n_features.
    y: array-like, shape (m_samples)
        Target array relative to X with m labels.
    axes: array of 3 axes
        matplotlib axes array to append the generated plot
    ylim: array
       
    '''
    if axes == None:
        _, axes = plt.subplots(1, 1, figsize=(5, 5))
        
    axes.set(title=title, ylim=ylim, xlabel='Training examples', ylabel=f'Metric: {scoring}')
    # Use learning_curve function from sklearn
    train_sizes, train_scores, test_scores, fit_times, _ = \
       learning_curve(estimator, X, y, scoring=scoring, return_times = True,
                      cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    # Compute useful metrics
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
#     fit_times_mean = np.mean(fit_times, axis=1)
#     fit_times_std = np.std(fit_times, axis=1)
    # Plot the learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1, color='r')
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                        test_scores_mean + test_scores_std, alpha=0.1, color='g')
    axes.plot(train_sizes, train_scores_mean, 'o-', color='r',
                label='Train score')
    axes.plot(train_sizes, test_scores_mean, 'o-', color='g',
                label='Cross-validation score')
    axes.legend(loc='lower right')
    

### Hyperparameters tunning: Grid Search

In [9]:
def run_grid_search(estimator, X_train, y_train, X_test, y_test, hyperparams,  cv_value=5, 
                    scoring='roc_auc', splitting='random', std_scale=False,
                   randomGS = False, n_iter=10):
    # Format the hyperparms
    hyperparams_dict = {'estimator__' + key: val for key, val in hyperparams.items()}
    
    if std_scale:
        # Create the Pipe
        scaler = StandardScaler()
        pipe = Pipeline([('scaler', scaler),
                         ('estimator', estimator)])
    else:
        pipe = estimator
    
    # Do Grid Search
    if randomGS:
        gs = RandomizedSearchCV(estimator = pipe,  param_distributions = hyperparams,
                     cv = cv_value, scoring = scoring, n_jobs = 6, refit = True, n_iter=n_iter)
    else:
        gs = GridSearchCV(estimator = pipe, param_grid = hyperparams,
                     cv = cv_value, scoring = scoring, n_jobs = 6, refit = True)
    # Train the model
    gs.fit(X_train, y_train) 

    # Predictions
    y_train_predict = gs.predict_proba(X_train)
    y_test_predict = gs.predict_proba(X_test)
    
    # Values to return
    n_train_mols    = y_train.shape[0]
    n_train_actives = y_train.sum()
    n_test_mols     = y_test.shape[0]
    n_test_actives  = y_test.sum()
    mean_cv_roc     = gs.best_score_
    train_roc       = roc_auc_score(y_train, y_train_predict[:, 1])
    test_roc        = roc_auc_score(y_test, y_test_predict[:, 1])
    best_params     = gs.best_params_

    # Print some values
    print(f'No. of molecules in train set: {n_train_mols}, with {n_train_actives} actives.')
    print(f'No. of molecules in test set: {n_test_mols}, with {n_test_actives} actives.')
    print('')
    print('*'*10, 'GRID SEARCH RESULTS', '*'*10)
    print('- Mean CV ROC-AUC:\t{:.3f}'.format(mean_cv_roc))
    print('- Train ROC-AUC:  \t{:.3f}'.format(train_roc))
    print('- Test ROC-AUC:   \t{:.3f}'.format(test_roc))
    print('- Best hyperparameters', best_params)
    print('**'*21)
    print('')
    
    return [n_train_mols, n_train_actives, n_test_mols, n_test_actives,
            mean_cv_roc, train_roc, test_roc, best_params]

### Function to report the Best Conformation's ROC-AUC for a given subset of samples

In [10]:
#************************************************************************
# Returns the best conformatio's ROC-AUC value of a given subset X and y
#************************************************************************

def get_roc_auc_DkSc(X_train, y_train, X_test, y_test, verbose=True):
    roc_auc_train = X_train.apply(
        lambda x: roc_auc_score(y_true= y_train, y_score= -x), axis=0)
    roc_auc_test = X_test.apply(
        lambda x: roc_auc_score(y_true= y_test, y_score= -x), axis=0)
    # Values to return
    train_best_roc = roc_auc_train.max()
    train_median   = roc_auc_train.median()
    train_mean     = roc_auc_train.mean()
    test_best_roc = roc_auc_test.max()
    test_median   = roc_auc_test.median()
    test_mean     = roc_auc_test.mean()
    
    if verbose:
        print("***** Best Conformation's ROC-AUC using docking scores *****")
        
        print("> Train best conf. ROC-AUC: {:.3f}".format(train_best_roc) +
              " \t> median: {:.3f}".format(train_median) +
              ', mean: {:.3f}'.format(train_mean))
        
        print("> Test best conf. ROC-AUC: {:.3f}".format(test_best_roc) +
              " \t> median: {:.3f}".format(test_median) +
              ', mean: {:.3f}'.format(test_mean))
        print('**'*32)
    # return a list of results to capture by the  wrapper function
    return [train_best_roc, train_median, train_mean,
            test_best_roc, test_median, test_mean]
        

In [11]:
#******************************************
# Decorator functions to capture GS results
#******************************************
from functools import wraps

def capture_GS_results(results_dict=None, capture=True):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if results_dict != None and capture:
                results = func(*args, **kwargs)
                # Create a key with the first four values
                key = '_'.join(results[:4])
                # Append results to the results dictionary
                results_dict[key] = results
            else:
                return func(*args, **kwargs)
        return wrapper
    return decorator
            
#******************************************************
# Dictionary to capture GS results using the decorator 
#******************************************************
results_dict = {}

#******************************************************
# Function to Split and run Grid Search
#******************************************************
@capture_GS_results(results_dict)
def split_and_gs(train_name, test_name, estimator_name,
                 X, y, estimator, hyperparams, splitting='random', 
                 test_size=0.25, scaffold_series=None, **kwargs):
    '''Given a X and y sets, a sklean estimator and an splitting method, 
    performs Grid Search CV using the parsed hyperparams'''
    #**************
    # Do the split
    #**************
    if splitting == 'scaffold':
        X_train, X_test, y_train, y_test = \
            train_test_scaffold_split(X, y, scaffold_series = scaffold_series,
                test_size=test_size, stratify=y)
    elif splitting == 'random':
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=test_size, stratify=y)
        
    # Verbose
    print(f'{estimator_name} => Train: {train_name}; Test: {test_name}; split: {splitting}')
    
    # Function to run Grid Search
    #----------------------------
    gs_results = run_grid_search(estimator, 
                    X_train, y_train, X_test, y_test, 
                    hyperparams = hyperparams,  **kwargs)

    # Function to extract ROC results from DkSc values 
    #-------------------------------------------------
    dksc_results = get_roc_auc_DkSc(X_train, y_train, X_test, y_test)
    
    # Return both list of results to captured by the decorador function
    return [train_name, test_name, estimator_name, splitting] + gs_results + dksc_results

In [12]:
results_dict

{}

#  Hyperparameter Tunning: Grid Search
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DEKOIS Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [13]:
#### library = 'DEKOIS'
library = 'DEKOIS'

# Train and test over DEKOIS
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [14]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.875
- Train ROC-AUC:  	0.885
- Test ROC-AUC:   	0.926
- Best hyperparameters {'C': 1e-08}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.894 	> median: 0.822, mean: 0.820
> Test best conf. ROC-AUC: 0.966 	> median: 0.866, mean: 0.863
****************************************************************
CPU times: user 904 ms, sys: 424 ms, total: 1.33 s
Wall time: 5.8 s


In [15]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
              X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.847
- Train ROC-AUC:  	0.875
- Test ROC-AUC:   	0.956
- Best hyperparameters {'C': 1e-08}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 617 ms, sys: 84.9 ms, total: 702 ms
Wall time: 1.17 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [16]:
%%time
estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7), 
               'gamma': np.geomspace(1e-8, 1e4, 7)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.896
- Train ROC-AUC:  	0.906
- Test ROC-AUC:   	0.876
- Best hyperparameters {'C': 1e-08, 'gamma': 0.0001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.906 	> median: 0.840, mean: 0.837
> Test best conf. ROC-AUC: 0.926 	> median: 0.816, mean: 0.815
****************************************************************
CPU times: user 1.63 s, sys: 366 ms, total: 1.99 s
Wall time: 11.8 s


In [17]:
%%time
estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7), 
               'gamma': np.geomspace(1e-8, 1e4, 7)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.846
- Train ROC-AUC:  	0.875
- Test ROC-AUC:   	0.953
- Best hyperparameters {'C': 1e-08, 'gamma': 1e-06}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.67 s, sys: 425 ms, total: 2.1 s
Wall time: 11.9 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [18]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.903
- Train ROC-AUC:  	0.902
- Test ROC-AUC:   	0.854
- Best hyperparameters {'C': 0.0001, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.918 	> median: 0.848, mean: 0.843
> Test best conf. ROC-AUC: 0.910 	> median: 0.802, mean: 0.800
****************************************************************
CPU times: user 1.18 s, sys: 550 ms, total: 1.73 s
Wall time: 11.4 s


In [19]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e4, 7), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.863
- Train ROC-AUC:  	0.869
- Test ROC-AUC:   	0.953
- Best hyperparameters {'C': 1e-08, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.18 s, sys: 601 ms, total: 1.78 s
Wall time: 11.3 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [20]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

kNN => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.892
- Train ROC-AUC:  	0.901
- Test ROC-AUC:   	0.828
- Best hyperparameters {'n_neighbors': 225, 'p': 2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.915 	> median: 0.846, mean: 0.843
> Test best conf. ROC-AUC: 0.891 	> median: 0.795, mean: 0.794
****************************************************************
CPU times: user 1.1 s, sys: 158 ms, total: 1.26 s
Wall time: 1.73 s


In [21]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

kNN => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.839
- Train ROC-AUC:  	0.871
- Test ROC-AUC:   	0.953
- Best hyperparameters {'n_neighbors': 225, 'p': 2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 848 ms, sys: 85 ms, total: 933 ms
Wall time: 1.29 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [22]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 915, with 30 actives.
No. of molecules in test set: 306, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.869
- Train ROC-AUC:  	0.889
- Test ROC-AUC:   	0.758
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.902 	> median: 0.834, mean: 0.829
> Test best conf. ROC-AUC: 0.921 	> median: 0.842, mean: 0.838
****************************************************************
CPU times: user 1.44 s, sys: 433 ms, total: 1.87 s
Wall time: 2.15 s


In [23]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 885, with 30 actives.
No. of molecules in test set: 336, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.820
- Train ROC-AUC:  	0.882
- Test ROC-AUC:   	0.918
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 3, 'max_features': 'log2', 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.884 	> median: 0.816, mean: 0.812
> Test best conf. ROC-AUC: 0.965 	> median: 0.897, mean: 0.893
****************************************************************
CPU times: user 1.22 s, sys: 301 ms, total: 1.52 s
Wall time: 1.98 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [24]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 21.2 ms, sys: 15 ms, total: 36.2 ms
Wall time: 545 ms


In [25]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 0 ns, sys: 37 µs, total: 37 µs
Wall time: 45.8 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [26]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: DEKOIS; Test: DEKOIS; split: random


KeyboardInterrupt: 

In [27]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: DEKOIS; Test: DEKOIS; split: scaffold


KeyboardInterrupt: 

***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DUD Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [28]:
library = 'DUD'

# Train and test over DUDU
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [28]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [ ]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e-2, 4), 
               'gamma': np.geomspace(1e-6, 1e0, 4)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [ ]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

In [ ]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

***

<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with different libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

##### Here, Random or Scaffold Splitting are not used

In [ ]:
# Train DEKOIS, test DUD
@capture_GS_results(results_dict)
def split_by_library_and_gs(train_name, test_name, estimator_name,
                            X, y, lib_train_name, lib_test_name, **kwargs):
    '''Train-Test "split" by library, and run Grid Search. Splits the main dataframe by library
    using different molecular libraries for Training and Testing.'''
    splitting='by_library'
    
    X_train = X.loc[lib_train_name]
    y_train = y.loc[lib_train_name]

    X_test = X.loc[lib_test_name]
    y_test = y.loc[lib_test_name]

    # Verbose
    print(f'{estimator_name} => Train: {train_name}; Test: {test_name}; split: {splitting}')
    
    # Function to run Grid Search
    #----------------------------
    gs_results = run_grid_search(estimator, 
                    X_train, y_train, X_test, y_test, 
                    hyperparams = hyperparams,  **kwargs)

    # Function to extract ROC results from DkSc values 
    #-------------------------------------------------
    dksc_results = get_roc_auc_DkSc(X_train, y_train, X_test, y_test)
    
    # Return both list of results to captured by the decorador function
    return [train_name, test_name, estimator_name, splitting] + gs_results + dksc_results

In [ ]:
# **********************************************
# Train and Test sets from diferent libraries
# **********************************************
X = X_merged_dksc
y = y_true_merged

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM  </h3>
<b>Train and Test with different libraries</b>

In [ ]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}
# Train DEKOIS, test DUD

train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

In [ ]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>Train and Test with different libraries</b>

In [ ]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

estimator_name = 'rbfSVC'
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e2, 6)}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

In [ ]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

estimator_name = 'rbfSVC'
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e2, 6)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Train and Test with different libraries</b>

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6),
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

In [ ]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest  </h3>
<b>Train and Test with different libraries</b>

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library

In [ ]:
# Train and test over 
X = X_merged_dksc
y = y_true_merged
library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.svm import SVC

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C':np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e4, 7)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: KKN - 1-NN as reference estimator </h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = '1-NN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [1], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = '1-NN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [1], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Random Forest </h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: X Gradient Boosting</h3>
<b>Merged Libraries</b>

In [ ]:
%%time

from xgboost import XGBClassifier

estimator_name = 'XGB_tree'
estimator = XGBClassifier()
hyperparams = {'n_estimators': [200, 300],
               'max_depth': [2, 3, 10, 20],
               'learning_rate': [0.05, 0.1],
               'gamma': [0.01, 0.1, 0.5, 1],
               'alpha': [0.01, 0.1, 0.5, 1],
               'subsample': [0.3, 0.5],
               'colsample_bytree': [0.3, 0.5, 1]
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None,
             # RandomizedGS
            randomGS=True, n_iter=50)

In [ ]:
%%time

from xgboost import XGBClassifier

estimator_name = 'XGB_tree'
estimator = XGBClassifier()
hyperparams = {'n_estimators': [200, 300],
               'max_depth': [2, 3, 10],
               'learning_rate': [0.05, 0.1],
               'gamma': [0.01, 0.1, 0.5, 1],
               'alpha': [0.01, 0.1, 0.5, 1],
               'subsample': [0.3, 0.5],
               'colsample_bytree': [0.3, 0.5, 1]
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series,
             # RandomizedGS
             randomGS=True, n_iter=50)

## Get, Format, and Save the Results

In [ ]:
### Save Results
row_names = ['Train_set', 'Test_set', 'Model name', 'Split', 
             'N_actives_train', 'N_actives_test', 'N_mols_train', 'Num_mols_test',
            'Mean-CV-ROC', 'ROC-AUC_train', 'ROC-AUC_test', 'best_params',
            'DkS_max_ROC_train',  'DkSc_med_ROC_train', 'DkSc_mean_ROC_train', 
            'DkS_max_ROC_test',  'DkSc_med_ROC_test', 'DkSc_mean_ROC_test']


# Load or save
save=True
filename='./GRID_SEARCH_CV_results.csv'
if save:
    results_df = pd.DataFrame(results_dict, index=row_names).T.reset_index().drop('index', axis=1)
    results_df.to_csv(filename)
else:
    results_df = pd.read_csv(filename)
    
results_df

***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries: Shuffle *y* target values in the train set
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library

In [ ]:
# Train and test over 
X = X_merged_dksc
# ***** Permutate y values *****
y = y_true_merged.sample(frac=1)

library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

In [ ]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.svm import SVC

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

In [ ]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e4, 3), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

In [ ]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e5, 4), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)